In [1]:
import h3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import numpy as np
import glob
import os
import datetime

import geopandas as gpd
import shapely
from shapely import wkt
from shapely import Polygon
from shapely.plotting import plot_polygon

from matplotlib.ticker import FuncFormatter


pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)

#Set color palette in code
colors = ['#8468F5','#EC74E7','#B2B2B2','#FFCB5C','#46B6E8','#2FD4A1','#333333']
sns.set_palette(sns.color_palette(colors), n_colors = 20)

# Read the pre-processed station data and the trips data

In [2]:
# Stations:
file_path = './bike_data/city-bike-stations-cleaned.csv'
df_stations = pd.read_csv(file_path)


# Bike trips:
directory_path = './bike_data/city-bike-trips-cleaned.csv'
df_trips = pd.read_csv(directory_path)

# Change the timestamps into datetime
df_trips['Departure'] = pd.to_datetime(df_trips['Departure'])
df_trips['Return'] = pd.to_datetime(df_trips['Return'])

df_trips['departure_date'] = pd.to_datetime(df_trips['departure_date'])
df_trips['return_date'] = pd.to_datetime(df_trips['return_date'])

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (25,26,27,28,32,33,34,35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Folium visualizations

## Define functions for plotting

In [3]:
from h3 import h3
import folium
import leafmap.foliumap as leafmap

center_lng = df_stations.lng.mean()
center_lat = df_stations.lat.mean()


def init_map(zoom=11):
    return folium.Map(location= [center_lat, center_lng], zoom_start=zoom, tiles="cartodbpositron")

def visualize_hexagons(hexagons, color="black", fill_color="purple", w=2, folium_map=None, fill_opacity=0.2):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = init_map()
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline, weight=w, color=color, fill_color=fill_color,
                                    fill_opacity=fill_opacity)
        m.add_child(my_PolyLine)
    return m

def visualize_points(lat, lng, color='black', folium_map=None, r=2):
    if folium_map is None:
        m = init_map()
    else:
        m = folium_map
        
    for i in range(0, len(lat)):
        scatter_marker = folium.CircleMarker(location=[lat[i], lng[i]], radius=r, color=color, fill_color=color)
        m.add_child(scatter_marker)
        
    return m
    

def visualize_polygon(polyline, color):
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    #m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    m = folium.Map(location=[center_lat, center_lng], zoom_start=13, tiles='openstreetmap')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
    m.add_child(my_PolyLine)
    return m

def visualize_line(start_lat, start_lng, end_lat, end_lng, color='red', w=2, folium_map=None):
    if folium_map is None:
        m = init_map()
    else:
        m = folium_map
        
    for i in range(0, len(start_lat)):
        line = folium.PolyLine(locations = [[start_lat[i], start_lng[i]], [end_lat[i], end_lng[i]]],
                               weight=w, color=color)
        m.add_child(line)
    return m

In [4]:
import folium
from geojson import Feature, Point, FeatureCollection
import json
import matplotlib
import branca
from branca.element import Element
import math

center_lng = df_stations.lng.mean()
center_lat = df_stations.lat.mean()

# Rounding functions to round the values to closest magnitude
def roundup(x, magnitude=100):
     return int(math.ceil(x / magnitude)) * magnitude
    
def rounddown(x, magnitude=100):
     return int(math.floor(x / magnitude)) * magnitude

# Taken from: https://gist.github.com/jens-andersson-2-wcar/7cecb40db1fbc051f2822262be42d11b

def hexagons_dataframe_to_geojson(df, hexa_col, column_name):
    """
    Produce the GeoJSON for a dataframe, constructing the geometry from the "hex_id" column
    and with a property matching the one in column_name
    """    
    list_features = []
    
    for i,row in df.iterrows():
        try:
            geometry_for_row = { "type" : "Polygon",
                                "coordinates": [h3.h3_to_geo_boundary(h=row[hexa_col],geo_json=True)]}
            feature = Feature(geometry = geometry_for_row , id = row[hexa_col],
                              properties = {column_name : row[column_name]})
            list_features.append(feature)
        except:
            print("An exception occurred for hex " + row[hexa_col]) 

    feat_collection = FeatureCollection(list_features)
    geojson_result = json.dumps(feat_collection)
    return geojson_result


def get_color(custom_cm, val, vmin, vmax):
    return matplotlib.colors.to_hex(custom_cm((val-vmin)/(vmax-vmin)))


def choropleth_map(df_aggreg, hexa_col, column_name, border_color = 'black', fill_opacity = 0.7,
                   color_map_name = "Purples", initial_map = None, min_value=None, max_value=None,
                   color_map_caption = None):
    """
    Creates choropleth maps given the aggregated data. initial_map can be an existing map to draw on top of.
    """    
    #colormap
    if min_value==None:
        min_value = df_aggreg[column_name].min()
    if max_value == None:
        max_value = df_aggreg[column_name].max()
        
            
    # the name of the layer just needs to be unique, put something silly there for now:
    name_layer = "Choropleth " + str(df_aggreg)
    
    if initial_map is None:
        initial_map = init_map()

    #create geojson data from dataframe
    geojson_data = hexagons_dataframe_to_geojson(df = df_aggreg, hexa_col=hexa_col, column_name=column_name)

    custom_cm = matplotlib.cm.get_cmap(color_map_name)

    folium.GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': get_color(custom_cm, feature['properties'][column_name], vmin=min_value, vmax=max_value),
            'color': border_color,
            'weight': 1,
            'fillOpacity': fill_opacity 
        }, 
        name = name_layer
    ).add_to(initial_map)
    
    # Create a colorbar legend
    color_min = get_color(custom_cm, min_value, vmin=min_value, vmax=max_value)
    color_max = get_color(custom_cm, max_value, vmin=min_value, vmax=max_value)
    color_scale = folium.LinearColormap([color_min, color_max], vmin=min_value, vmax=max_value)
    # Add the colorbar legend to the map
    if color_map_caption == None:
        color_scale.caption = column_name
    else:
        color_scale.caption = color_map_caption
    initial_map.add_child(color_scale)

        

    return initial_map

def add_categorical_legend(m, plot_colors, indexes, text):
    ticks = np.add(indexes, 0.5).tolist()
    colormap = branca.colormap.StepColormap(colors=plot_colors, index=indexes, tick_labels= ticks,
                                            vmin=indexes[0], vmax=indexes[-1]+1)
    colormap.caption = text
    
    m.add_child(colormap)
    e = Element("""
                var ticks = document.querySelectorAll('div.legend g.tick text')
                for(var i = 0; i < ticks.length; i++) {
                    var value = parseFloat(ticks[i].textContent.replace(',', ''))
                    var newvalue = Math.floor(value).toFixed(0).toString()
                    ticks[i].textContent = newvalue
                }
                """)
    html = colormap.get_root()
    html.script.get_root().render()
    html.script.add_child(e)


## Visualize the data

In [5]:
# Plot the station and the year they were added to the city bike network
m = init_map(zoom=11.5)
use_data = df_stations[df_stations.Year==2016]
visualize_points(use_data.lat.tolist(), use_data.lng.tolist(), folium_map=m, color=colors[0])
use_data = df_stations[df_stations.Year==2017]
visualize_points(use_data.lat.tolist(), use_data.lng.tolist(), folium_map=m, color=colors[1])
use_data = df_stations[df_stations.Year==2018]
visualize_points(use_data.lat.tolist(), use_data.lng.tolist(), folium_map=m, color=colors[2])
use_data = df_stations[df_stations.Year==2019]
visualize_points(use_data.lat.tolist(), use_data.lng.tolist(), folium_map=m, color=colors[3])
use_data = df_stations[df_stations.Year==2020]
visualize_points(use_data.lat.tolist(), use_data.lng.tolist(), folium_map=m, color=colors[4])
use_data = df_stations[df_stations.Year==2021]
visualize_points(use_data.lat.tolist(), use_data.lng.tolist(), folium_map=m, color=colors[5])
add_categorical_legend(m, [colors[0], colors[1], colors[2], colors[3], colors[4], colors[5]], 
                       [2016, 2017, 2018, 2019, 2020, 2021], 'Year station was added')

m.save("images/station_year.html")
display(m)

In [6]:
# Plot the average covered distance by the return station
col_hexa = 'return_h3_res8'
col_by = 'Covered distance (m)'

#num_trips = df_trips.groupby([col_hexa,'return_date'])[[col_by]].mean().reset_index()
num_trips = df_trips.groupby([col_hexa])[[col_by]].mean().reset_index()

plot_map = choropleth_map(num_trips, hexa_col=col_hexa, column_name = 'Covered distance (m)',
                          min_value=rounddown(num_trips[col_by].min()), max_value=roundup(num_trips[col_by].max()),
                          fill_opacity=0.9,
                          color_map_caption = 'Average covered distance (m) by return station')

plot_map.save("images/covered_distance_byhexagon.html")
display(plot_map)

In [7]:
m = init_map(zoom=11.5)
visualize_hexagons(df_stations.h3_res8.unique().tolist(), color="black", fill_color=None, w=0.6, folium_map=m)
visualize_hexagons(df_stations.h3_res7.unique().tolist(), color="gray", fill_color=None, w=3, folium_map=m)
visualize_hexagons(df_stations.h3_res6.unique().tolist(), color="dimgray", fill_color=None, w=4, folium_map=m)

visualize_points(df_stations.lat.tolist(), df_stations.lng.tolist(), folium_map=m, color=colors[0], r=2)

m.save("image/hexagons_at_different_resolutions.html")

display(m)

In [8]:
# Lat and lng for the return and departure hexagons
selected_hexagon = '871126d33ffffff'
hexa = h3.h3_to_geo(selected_hexagon)

m = init_map(zoom=11.5)

visualize_points(df_stations.lat.tolist(), df_stations.lng.tolist(), folium_map=m, color=colors[0], r=1)
visualize_hexagons(df_stations.h3_res7.unique().tolist(), color="black", fill_color=None, w=0.6, folium_map=m)
visualize_hexagons([selected_hexagon], color='black', fill_color=colors[0], w=2, folium_map=m, fill_opacity=0.7)

display(m)

In [9]:
col_hexa = 'h3_res8'
col_by = 'ID'

num_stations = df_stations.groupby([col_hexa])[[col_by]].count().reset_index()

choropleth_map(num_stations, hexa_col=col_hexa, column_name = 'ID', min_value=0, color_map_name ='coolwarm',
               fill_opacity=0.9, color_map_caption = 'Number of stations per hexagon')

In [10]:
col_hexa = 'departure_h3_res8'
col_by = "Departure"

num_trips = df_trips.groupby([col_hexa,'departure_date'])[[col_by]].count().reset_index()
num_trips = num_trips.groupby([col_hexa])[[col_by]].mean().reset_index()

choropleth_map(num_trips, hexa_col=col_hexa, column_name = col_by, color_map_name ='coolwarm',
               fill_opacity=0.9, color_map_caption = 'Average number of daily departures per hexagon',
               min_value=0, max_value=roundup(num_trips[col_by].max(),50))

# Own trips

In [11]:
my_trips = df_trips[ (df_trips['Departure station name'] == 'Töölönlahdenkatu') & 
                    (df_trips['Return station name'] == 'Syystie')]

my_trips['Covered distance (km)'] = my_trips['Covered distance (m)']/1000
my_trips['Duration (min.)'] = my_trips['Duration (sec.)']/60
my_trips['Day of the week'] = my_trips['departure_day_name']

my_trips[['Departure', 'Return', 'Departure station name', 'Return station name',
          'Covered distance (km)', 'Duration (min.)', 'Day of the week']]

,Departure,Return,Departure station name,Return station name,Covered distance (km),Duration (min.),Day of the week
5575303,2021-08-13 17:22:04,2021-08-13 18:09:32,Töölönlahdenkatu,Syystie,12.489,47.383333,Friday
5721690,2021-08-04 16:37:40,2021-08-04 17:25:33,Töölönlahdenkatu,Syystie,12.078,47.866667,Wednesday
11059403,2021-07-10 17:47:51,2021-07-10 18:37:58,Töölönlahdenkatu,Syystie,12.078,50.100000,Saturday


In [12]:
m = visualize_line(my_trips.departure_lat.tolist(), my_trips.departure_lng.tolist(),
               my_trips.return_lat.tolist(), my_trips.return_lng.tolist(), color='black', w=1, folium_map=None)
visualize_points(df_stations.lat.tolist(), df_stations.lng.tolist(), folium_map=m, color=colors[0], r=1)
visualize_points(my_trips.departure_lat.tolist(), my_trips.departure_lng.tolist(), folium_map=m,
                 color='red', r=4)
visualize_points(my_trips.return_lat.tolist(), my_trips.return_lng.tolist(), folium_map=m,
                 color='red', r=4)

display(m)

In [13]:
departure_id = my_trips.departure_h3_res7.unique()[0]
return_id = my_trips.return_h3_res7.unique()[0]

similar_trips = df_trips[ (df_trips.departure_h3_res7 == departure_id) & (df_trips.return_h3_res7 == return_id)]
similar_trips['Covered distance (km)'] = similar_trips['Covered distance (m)']/1000
similar_trips['Duration (min.)'] = similar_trips['Duration (sec.)']/60
similar_trips['Day of the week'] = similar_trips['departure_day_name']

similar_trips[['Departure', 'Return', 'Covered distance (km)', 'Duration (min.)', 'Day of the week']]

,Departure,Return,Covered distance (km),Duration (min.),Day of the week
4732832,2021-09-04 22:24:12,2021-09-04 23:28:53,12.141,64.600000,Saturday
5575303,2021-08-13 17:22:04,2021-08-13 18:09:32,12.489,47.383333,Friday
5675659,2021-08-06 23:35:58,2021-08-08 12:10:09,4.128,2194.116667,Friday
5721690,2021-08-04 16:37:40,2021-08-04 17:25:33,12.078,47.866667,Wednesday
9518793,2021-06-28 10:17:37,2021-06-28 10:27:35,3.394,9.950000,Monday
9826518,2021-06-12 00:37:40,2021-06-12 01:27:09,12.055,49.416667,Saturday
11059403,2021-07-10 17:47:51,2021-07-10 18:37:58,12.078,50.100000,Saturday
11111635,2021-07-07 22:33:09,2021-07-07 23:17:39,11.286,44.400000,Wednesday
11133208,2021-07-06 23:23:34,2021-07-07 00:24:30,11.177,60.850000,Tuesday
11133211,2021-07-06 23:22:21,2021-07-07 00:20:33,11.284,58.116667,Tuesday


In [14]:
# Lat and lng for the return and departure hexagons
ret = h3.h3_to_geo(return_id)
dep = h3.h3_to_geo(departure_id)

m = visualize_line([dep[0]], [dep[1]], [ret[0]], [ret[1]], color='black', w=2, folium_map=None)
visualize_points(df_stations.lat.tolist(), df_stations.lng.tolist(), folium_map=m, color=colors[0], r=1)
visualize_points([dep[0]], [dep[1]], folium_map=m, color='red', r=2)
visualize_points([ret[0]], [ret[1]], folium_map=m, color='red', r=2)
visualize_hexagons([return_id], color="black", fill_color=colors[3], w=1.5, folium_map=m)
visualize_hexagons([departure_id], color="black", fill_color=colors[4], w=1.5, folium_map=m)


display(m)